In [1]:
import os
import shutil
import json
from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from run_experiments import run_experiments, Result

In [2]:
# set envvars
os.environ["OS_MEMORY_ALLOC_OVERRIDE"] = "25000000"
os.environ["ENGINE_PATH"] = "./target/x86_64-apple-darwin/release/hash_engine"
os.environ["RUST_LOG"] = "info"

In [3]:
# we need cwd to be the engine folder
os.chdir("./..")

In [ ]:
# clear the output folders
try:
    shutil.rmtree('./output')
    shutil.rmtree('./parts')
except FileNotFoundError:
    pass

In [5]:
# results = run_experiments(
#     project_paths=[Path("./experiments/experiment-test")], 
#     run_all_experiments=True,
#     cli_run_override="./target/x86_64-apple-darwin/release/cli",
#     build_args=["--no-default-features"],
#     cli_args=[]
# )

In [6]:
# results = run_experiments(
#     project_paths=[
#         Path("./experiments/sugarscape"), 
#         Path("./experiments/age"), 
#         Path("./experiments/neighbors"), 
#         Path("./experiments/empty_sim"), 
#         Path("./experiments/warehouse-logistics")], 
#     run_all_experiments=True,
#     cli_run_override="./target/x86_64-apple-darwin/release/cli",
#     build_args=["--no-default-features"],
#     cli_args=[],
#     continue_on_fail=True
# )

In [7]:
results = run_experiments(
    project_paths=[Path("./experiments/sugarscape")], 
    run_all_experiments=False,
    cli_run_override="./target/x86_64-apple-darwin/release/cli",
    build_args=["--no-default-features"],
    cli_args=["single-run", "--num-steps", "10"]
)

   Compiling provider v0.0.0 (/Users/alfie/Documents/code/hash/packages/engine/lib/provider)
   Compiling hash_engine v0.0.0 (/Users/alfie/Documents/code/hash/packages/engine)
   --> src/datastore/storage/visitor.rs:171:12
    |
171 |     pub fn shrink_with_data_length(&mut self, size: usize) -> Result<BufferChange> {
    |            ^^^^^^^^^^^^^^^^^^^^^^^
    |
    = note: `#[warn(dead_code)]` on by default

   Compiling cli v0.0.0 (/Users/alfie/Documents/code/hash/packages/engine/bin/cli)
   Compiling server v0.0.0 (/Users/alfie/Documents/code/hash/packages/engine/bin/server)
    Finished release [optimized] target(s) in 38.30s


Running Experiment with cmd: ./target/x86_64-apple-darwin/release/cli -p experiments/sugarscape single-run --num-steps 10
Time taken: 2.167445421218872
max_vms_memory: 69608022016
max_rss_memory: 6066176
Running Experiment succeeded


In [8]:
for experiment_group, experiment_results in results.items():
    print(f"Experiment Group: {experiment_group}")
    
    for experiment_name, experiment_result in experiment_results.items():
        print(f"Experiment Name: {experiment_name}")
        if experiment_result.res != Result.SUCCESS:
            print(f"Experiment failed, skipping")
            continue
        
        for output_folder in experiment_result.output_folders:
            global_properties = json.loads((output_folder / "globals.json").read_text())
            analysis_outputs = json.loads((output_folder / "analysis_outputs.json").read_text())
            state = json.loads((output_folder / "json_state.json").read_text())
            
            if not analysis_outputs['manifest'] and not analysis_outputs['buffers']:
                continue
            
            plot_defs = { plot_def['title']: (lambda d: d.pop('title') and d)(plot_def) for plot_def in analysis_outputs['manifest']['plots'] }
            outputs = analysis_outputs['buffers']
            
            plot(plot_defs, outputs)

Experiment Group: experiments/sugarscape
Experiment Name: single_run-e3ffa8
